<a href="https://colab.research.google.com/github/ko-bbie/bert_livedoor_classifier/blob/master/notebook/livedoor_bert_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 675kB 9.2MB/s 
     |████████████████████████████████| 3.8MB 24.5MB/s 
     |████████████████████████████████| 890kB 39.3MB/s 
     |████████████████████████████████| 1.1MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=252cc03107cc9d70d4c1c11090fabd70d993cfc1607bc3a4bfd653740ada6ab1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [ ]:
from google.colab import drive

# Google Driveをマウントする仮想マシン上のディレクトリ
DIR_DRIVE = './gdrive/'

# Google Drive上でのNotebook等の各種ファイルのパス
DIR_COLAB = DIR_DRIVE + 'My Drive/Colab Notebooks/'
DIR_PROJCET = DIR_COLAB + 'livedoor_classification/'

# Google Driveをマウント
drive.mount(DIR_DRIVE)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive/


In [ ]:
DATA_DIR = DIR_DRIVE + 'My Drive/nlp/data/'

In [ ]:
import pandas as pd
import pandas_profiling
import torch
import torch.nn as nn
import transformers
import re
import unicodedata
from tqdm import tqdm

In [ ]:
MAX_LEN = 256
TEST_BATCH_SIZE = 8
BERT_PATH = "cl-tohoku/bert-base-japanese-whole-word-masking"
MODEL_PATH = DATA_DIR + "wwm-model.bin"
TEST_FILE = DATA_DIR + "test.csv"
TOKENIZER = transformers.AutoTokenizer.from_pretrained(BERT_PATH)

In [ ]:
def category_dict():
    dic = {
        "sports-watch": [1, 0, 0, 0, 0, 0, 0, 0, 0],
        "smax": [0, 1, 0, 0, 0, 0, 0, 0, 0],
        "dokujo-tsushin": [0, 0, 1, 0, 0, 0, 0, 0, 0],
        "movie-enter": [0, 0, 0, 1, 0, 0, 0, 0, 0],
        "it-life-hack": [0, 0, 0, 0, 1, 0, 0, 0, 0],
        "kaden-channel": [0, 0, 0, 0, 0, 1, 0, 0, 0],
        "peachy": [0, 0, 0, 0, 0, 0, 1, 0, 0],
        "topic-news": [0, 0, 0, 0, 0, 0, 0, 1, 0],
        "livedoor-homme": [0, 0, 0, 0, 0, 0, 0, 0, 1],
    }
    return dic


def category_dict_from_num():
    dic = {
        0: "sports-watch",
        1: "smax",
        2: "dokujo-tsushin",
        3: "movie-enter",
        4: "it-life-hack",
        5: "kaden-channel",
        6: "peachy",
        7: "topic-news",
        8: "livedoor-homme",
    }
    return dic


def normalize(text):
    text = normalize_unicode(text)
    text = capitalize_symbols(text)
    return text


def normalize_unicode(text, form="NFKC"):
    normalized_text = unicodedata.normalize(form, text)
    normalized_text = re.sub(r"\s", " ", normalized_text)
    return normalized_text


def capitalize_symbols(text):
    # this function is often used for mecab morph analysis
    text = text.replace("・", "")
    table = str.maketrans(dict(zip("!?(),.:;/@%&[]", "！？（）、。：；／＠％＆［］")))
    capitalized_text = text.translate(table)
    return capitalized_text

In [ ]:
class LivedoorDataset:
    def __init__(self, article, targets):
        self.article = article
        self.targets = targets
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        self.category_dic = category_dict()

    def __len__(self):
        return len(self.article)

    def __getitem__(self, item):
        article = normalize(self.article[item])

        inputs = self.tokenizer.encode_plus(
            article, add_special_tokens=True, max_length=self.max_len
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        padding_len = self.max_len - len(ids)
        if padding_len > 0:
            ids = ids + [0] * padding_len
            mask = mask + [0] * padding_len
            token_type_ids = token_type_ids + [0] * padding_len

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(
                self.category_dic.get(self.targets[item]), dtype=torch.long
            ),
            "orig_targets": self.targets[item],
        }

In [ ]:
class BertBaseJapanese(nn.Module):
    def __init__(self):
        super(BertBaseJapanese, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 9)

    def forward(self, ids, mask, token_type_ids):
        _, pooled_output = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids
        )
        bo = self.bert_drop(pooled_output)
        return self.out(bo)

In [ ]:
def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []

    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.long)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        fin_targets.extend(targets.argmax(1).cpu().detach().numpy().tolist())
        fin_outputs.extend(outputs.argmax(1).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
def predict():
    df = pd.read_csv(TEST_FILE).fillna("none")

    test_dataset = LivedoorDataset(df.article.values, df.category.values)
    test_data_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=TEST_BATCH_SIZE, num_workers=4
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BertBaseJapanese()
    model.to(device)

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    model.load_state_dict(torch.load(MODEL_PATH))
    outputs, _ = eval_fn(test_data_loader, model=model, device=device)

    return outputs

In [ ]:
outputs = predict()
dic = category_dict_from_num()
outputs = [dic.get(output)  for output in outputs]

In [ ]:
result = pd.read_csv(TEST_FILE)
result["pred_category"] = outputs

In [ ]:
result.head()

,textID,article,category,pred_category
0,3109,夏のデートの定番といえばやっぱりドライブ。海、山、夏フェスと、さまざまなレジャースポットに向...,livedoor-homme,peachy
1,3006,アルマーニ グループとエマール・プロパティーズPJSCが、アルマーニ ホテル＆リゾートの2...,livedoor-homme,peachy
2,498,みなさま、初めまして、じゃなくて二度目まして。 ゆるっとcafeで、東京の銭湯を楽しむマンガ...,dokujo-tsushin,dokujo-tsushin
3,4809,海外旅行に行くなら、混雑のピークを過ぎて旅行代金が安くなるこれからがおトク。「夏休みはこれか...,peachy,peachy
4,7031,19日に「日経トレンディNET」に掲載された記事「カシオの不人気デジカメが台湾や中国で大ブレ...,topic-news,topic-news


In [ ]:
result.to_csv(DATA_DIR + "result-wwm.csv", index=False)

## test

In [ ]:
test = result.category.values.tolist()
pred = result.pred_category.values.tolist()
correct = 0
for el1, el2 in zip(test, pred):
  if el1 == el2:
    correct += 1
print(f"Accuracy = {correct / len(pred)}")

Accuracy = 0.921302578018996
